In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Đọc file dữ liệu "data_banknote_authentication.txt" vào DataFrame df
df = pd.read_csv("../Data/data_banknote_authentication.txt", header=0)

# Gán tên cột cho DataFrame: "variance", "skewness", "curtosis", "entropy", "class"
df.columns = ["variance", "skewness", "curtosis", "entropy", "class"]

# Tách dữ liệu thành ma trận đặc trưng X (gồm 4 cột đầu tiên) và vector nhãn y (cột "class")
X = df.iloc[:, :-1].values
y = df["class"].values

# Chia tập dữ liệu thành tập huấn luyện (train) và tập kiểm tra (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Figure: Thiết lập kích thước biểu đồ
plt.figure(figsize=(10, 6))

# Histogram: Phân bố dữ liệu mẫu Genuine (nhãn = 0)
plt.hist(X_train[y_train == 0].flatten(), bins=30, alpha=0.7, label="Genuine", color='blue')

# Histogram: Phân bố dữ liệu mẫu Forged (nhãn = 1)
plt.hist(X_train[y_train == 1].flatten(), bins=30, alpha=0.7, label="Forged", color='red')

# Title: Tiêu đề của biểu đồ
plt.title("Phân bố dữ liệu ban đầu")

# X-axis Label: Nhãn trục X
plt.xlabel("Giá trị đặc trưng")

# Y-axis Label: Nhãn trục Y
plt.ylabel("Tần số")

# Legend: Chú thích phân biệt các lớp
plt.legend()
plt.show()

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA 2D: Giảm số chiều dữ liệu xuống còn 2 thành phần chính
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_train_scaled)

# Figure: Thiết lập kích thước biểu đồ
plt.figure(figsize=(8, 6))

# Scatter: Dữ liệu mẫu Genuine (nhãn = 0) sau PCA
plt.scatter(X_pca_2d[y_train == 0, 0], X_pca_2d[y_train == 0, 1], label="Genuine", alpha=0.7, c='blue')

# Scatter: Dữ liệu mẫu Forged (nhãn = 1) sau PCA
plt.scatter(X_pca_2d[y_train == 1, 0], X_pca_2d[y_train == 1, 1], label="Forged", alpha=0.7, c='red')

# Title: Tiêu đề của biểu đồ
plt.title("Dữ liệu Banknote Authentication")

# X-axis Label: Nhãn trục X
plt.xlabel("Principal Component 1")

# Y-axis Label: Nhãn trục Y
plt.ylabel("Principal Component 2")

# Legend: Chú thích phân biệt các lớp
plt.legend()

# Show: Hiển thị biểu đồ
plt.show()


In [ ]:
# PCA 3D: Giảm số chiều dữ liệu xuống còn 3 thành phần chính
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_train_scaled)

# Figure: Thiết lập kích thước biểu đồ 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter: Dữ liệu mẫu Genuine (nhãn = 0) sau PCA 3D
ax.scatter(X_pca_3d[y_train == 0, 0], X_pca_3d[y_train == 0, 1], X_pca_3d[y_train == 0, 2], label="Genuine", alpha=0.7, c='blue')

# Scatter: Dữ liệu mẫu Forged (nhãn = 1) sau PCA 3D
ax.scatter(X_pca_3d[y_train == 1, 0], X_pca_3d[y_train == 1, 1], X_pca_3d[y_train == 1, 2], label="Forged", alpha=0.7, c='red')

# Title: Tiêu đề của biểu đồ
ax.set_title("Dữ liệu Banknote Authentication")

# X-axis Label: Nhãn trục X
ax.set_xlabel("Principal Component 1")

# Y-axis Label: Nhãn trục Y
ax.set_ylabel("Principal Component 2")

# Z-axis Label: Nhãn trục Z
ax.set_zlabel("Principal Component 3")

# Legend: Chú thích phân biệt các lớp
ax.legend()
plt.show()

In [ ]:
# Linear Discriminant Analysis

# Mean: Tính trung bình của từng lớp
mean_0 = np.mean(X_train_scaled[y_train == 0], axis=0)
mean_1 = np.mean(X_train_scaled[y_train == 1], axis=0)

# Within-class Scatter (SW): Tính ma trận scatter trong lớp
SW = np.zeros((X.shape[1], X.shape[1]))
for i, mean in zip([0, 1], [mean_0, mean_1]):
    X_class = X_train_scaled[y_train == i]
    SW += (X_class - mean).T @ (X_class - mean)

# Between-class Scatter (SB): Tính ma trận scatter giữa các lớp
SB = np.outer((mean_0 - mean_1), (mean_0 - mean_1))

# Optimal Vector: Tính vector tối ưu
eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(SW) @ SB)
w = eigen_vectors[:, np.argmax(eigen_values)].real.reshape(-1, 1)

# Project: Chiếu dữ liệu lên vector LDA
X_train_lda = X_train_scaled @ w
X_test_lda = X_test_scaled @ w

In [ ]:
# Visualization LDA
title = "Phân bố sau khi thực hiện Linear Discriminant Analysis"
plt.figure(figsize=(8, 6))
plt.hist(X_train_lda[y_train == 0], bins=20, alpha=0.7, label="Genuine", color='blue')
plt.hist(X_train_lda[y_train == 1], bins=20, alpha=0.7, label="Forged", color='red')
plt.title(title)
plt.xlabel("Giá trị chiếu theo trục phân biệt LDA")
plt.ylabel("Tần số")
plt.legend()
plt.show()

In [ ]:
# 1D scatter / strip plot
plt.figure(figsize=(10, 2))
plt.scatter(X_train_lda[y_train == 0], np.zeros_like(X_train_lda[y_train == 0]), 
            color='blue', alpha=0.6, label="Genuine", marker='|', s=100)
plt.scatter(X_train_lda[y_train == 1], np.zeros_like(X_train_lda[y_train == 1]), 
            color='red', alpha=0.6, label="Forged", marker='|', s=100)

plt.title("1D Scatter Plot sau LDA")
plt.xlabel("Giá trị chiếu theo trục phân biệt LDA")
plt.yticks([])
plt.legend(loc='upper right')
plt.grid(True, axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.kdeplot(X_train_lda[y_train == 0].flatten(), label="Genuine", color='blue', fill=True, alpha=0.5)
sns.kdeplot(X_train_lda[y_train == 1].flatten(), label="Forged", color='red', fill=True, alpha=0.5)

plt.title("Phân bố theo KDE sau khi thực hiện LDA")
plt.xlabel("Giá trị chiếu theo trục phân biệt LDA")
plt.ylabel("Mật độ xác suất")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Mean in LDA Space: Tính trung bình của từng lớp sau khi chiếu LDA
m_0_lda = np.mean(X_train_lda[y_train == 0])
m_1_lda = np.mean(X_train_lda[y_train == 1])

# Nearest Centroid Classifier: Hàm phân loại theo khoảng cách tới trung tâm
def nearest_centroid_classifier(X_lda, m_0, m_1):
    return np.where(np.abs(X_lda - m_0) < np.abs(X_lda - m_1), 0, 1)

# Prediction: Dự đoán nhãn cho tập test
y_pred = nearest_centroid_classifier(X_test_lda, m_0_lda, m_1_lda)

# Accuracy: Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"LDA Accuracy on test set: {accuracy:.2f}%")

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Random Test Case: Chọn ngẫu nhiên một mẫu từ tập test
random_idx = random.randint(0, len(X_test_lda) - 1)
original_test_case = X_test[random_idx]
test_case = X_test_lda[random_idx]
true_label = y_test[random_idx]

# Prediction: Dự đoán nhãn cho test case
predicted_label = nearest_centroid_classifier(test_case, m_0_lda, m_1_lda)

# Print Info: In ra thông tin test case
print(original_test_case)
print(test_case)
print("Nhãn thực tế:", true_label)
print("Nhãn dự đoán:", predicted_label[0])

# Random Another Test Case: Chọn ngẫu nhiên thêm một mẫu khác
random_idx = random.randint(0, len(X_test_lda) - 1)
test_case = X_test_lda[random_idx]
true_label = y_test[random_idx]
predicted_label = nearest_centroid_classifier(test_case, m_0_lda, m_1_lda)[0]

# Figure: Thiết lập biểu đồ
plt.figure(figsize=(10, 2))
plt.title("Biểu diễn dữ liệu sau khi chiếu LDA với test case", fontsize=14)

# Scatter: Dữ liệu huấn luyện lớp Genuine
plt.scatter(X_train_lda[y_train == 0], np.zeros_like(X_train_lda[y_train == 0]), 
            color='blue', label='Genuine', alpha=0.5)

# Scatter: Dữ liệu huấn luyện lớp Forged
plt.scatter(X_train_lda[y_train == 1], np.zeros_like(X_train_lda[y_train == 1]), 
            color='red', label='Forged', alpha=0.5)

# Scatter: Điểm test case
plt.scatter(test_case, 0, color='gold', edgecolor='black', s=200, label='Test case')

# Scatter: Trung tâm hai lớp
plt.scatter(m_0_lda, 0, color='orange', edgecolor='black', s=250, label='Trung tâm Genuine')
plt.scatter(m_1_lda, 0, color='purple', edgecolor='black', s=250, label='Trung tâm Forged')

# Format: Tắt trục Y, đặt nhãn trục X, thêm chú thích và lưới
plt.yticks([])
plt.xlabel("Giá trị chiếu theo trục phân biệt LDA")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()